<a href="https://colab.research.google.com/github/IgorTFromK/GoogleSpeechCommandClassifier/blob/master/train_le_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import time

FILE_DIR_MAIN = "/content/google_speech_command/"
FILE_DIR_GRIDSEARCH = "/content/google_speech_command/gridsearch/"
FILE_DIR_TRAIN = FILE_DIR_MAIN + "train/"
FILE_DIR_TEST = FILE_DIR_MAIN + "test/"
FILE_DIR_EVAL = FILE_DIR_MAIN + "eval/"



In [ ]:
def create_model():
  pass

In [ ]:
def train_model():
  pass

In [ ]:
def test_model():
  pass

In [ ]:
def long_op():
  for x in range(1000):
    time.sleep(2.4)

In [ ]:
long_op()

In [ ]:
def write_file_test(filename:str):
  with open(filename, 'w') as writefile:
    for x in range(10):
      writefile.write("Line: {:d}\n".format(x))

In [ ]:
def read_file_test(filename: str):
  with open(filename, 'r') as readfile:
    for line in readfile:
      print(line)